<img src="../images/QISKit-c.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="250 px" align="left">

## _*Basic Quantum Gates and Their Mathematical Definitions*_ 

The latest version of this notebook is available on https://github.com/QISKit/qiskit-tutorial.

For more information about how to use the IBM Q experience (QX), consult the [tutorials](https://quantumexperience.ng.bluemix.net/qstage/#/tutorial?sectionId=c59b3710b928891a1420190148a72cce&pageIndex=0), or check out the [community](https://quantumexperience.ng.bluemix.net/qstage/#/community).

***
### Contributors
Rudy Raymond

## Quantum Gates 

Quantum gates on $q$ qubits can be regarded as unitary operations rotating a complex vector of dimension $2^q$, that corresponds to the quantum state. This tutorial is written to list elementary gates and their corresponding unitary matrices. Each gate is documented in [the openqasm](https://github.com/QISKit/openqasm/blob/master/spec/qasm2.pdf). The gates are defined in [the file `qelib1.inc`](https://github.com/QISKit/openqasm/blob/master/examples/generic/qelib1.inc), and implemented in [the standard extension](https://github.com/QISKit/qiskit-sdk-py/tree/master/qiskit/extensions/standard). The gates can be divided into one-qubit gates and multi-qubit gates. 

In the hereafter, following the standard in the quantum community we treat the order of the qubits from left to right, namely, the least significant bit (LSB) is the left-most qubit. Thus, the tensor product of $q_0$, $q_1$, and $q_2$ of the three qubits is defined as $q_0 \otimes q_1 \otimes q_2$. 

Notice that the IBM Quantum experience uses order the qubits in the opposite way (the tensor product is $q_2 \otimes q_1 \otimes q_0$). Thus to get the standard order, we must store the measurement of the first qubit to the last bit, the second qubit to the second last bit, and so on. 

We start with preparing the environment. 

In [2]:
import sys
if sys.version_info < (3,0):
    raise Exception("Please use Python version 3 or greater.")

# useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi

#Uncomment the line below and set the path to qiskit-sdk-py if you do not install qiskit-sdk with pip
#sys.path.append("../../qiskit-sdk-py/")

# importing the QISKit
from qiskit import QuantumProgram
import Qconfig

# import basic plot tools
from qiskit.tools.visualization import plot_histogram

Q_program = QuantumProgram()
Q_program.set_api(Qconfig.APItoken, Qconfig.config["url"]) # set the APIToken and API url

# Creating registers
# prepare 3 qubits
qr = Q_program.create_quantum_register("qr", 3)
# 3 bits for recording the measurement of the qubits
cr = Q_program.create_classical_register("cr", 3)

circuitName = "gatesDemo"
demoCircuit = Q_program.create_circuit(circuitName, [qr], [cr])

## One-Qubit Gates

The one-qubit gates available consist of Pauli $X, Y, Z$ gates, the Identity gate, primitives, and standard rotation gates. The primitives contain parametrized $u$ matrices (from one up to three parameters),  the Hadamard gate, the square-root of $Z$ (also referred as the $S$) gate and its conjugate, the square-root of $S$ (also referred as the $T$) gate and its conjugate. The standard rotation gates are those that define rotations around standard axis.  

### Pauli gates

#### $X$: bit-flip gate

The bit-flip gate $X$ is defined as:

$$
X = u3(\pi, 0, pi) = 
\begin{pmatrix}
0 & 1\\
1 & 0
\end{pmatrix}
$$

#### $Y$: bit- and phase-flip gate

The $Y$ gate is defined as:

$$
Y = u3(\pi, \pi/2, \pi/2) = 
\begin{pmatrix}
0 & -i\\
i & 0
\end{pmatrix}
$$

#### $Z$: phase-flip gate

The $Z$ gate is defined as:

$$
Z = u1(\pi) = 
\begin{pmatrix}
1 & 0\\
0 & -1
\end{pmatrix}
$$

The pauli gates can be added as below.

In [3]:
demoCircuit.x(qr[0]) #applying  x gate to the first qubit
demoCircuit.y(qr[0]) #applying  y gate to the first qubit
demoCircuit.z(qr[0]) #applying  z gate to the first qubit

### Identity gate

The identity gate does nothing (but can add noises in the real device).

$$
I = 
\begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix}.
$$

It can be added as below.

In [4]:
demoCircuit.iden(qr[0]) #identity gate on the first qubit

### Primitives

The primitives contain parametrized $u$ matrices (from one up to three parameters),  the Hadamard gate, the square-root of $Z$ (also referred as the $S$) gate and its conjugate, the square-root of $S$ (also referred as the $T$) gate and its conjugate. 

#### u3 (or,  U gates with three parameters)

$u3(\theta, \phi, \lambda$ (or, $U(\theta, \phi, \lambda)$) gates are rotations of 1-qubit state whose mathematical definition are:

$$
u3(\theta, \phi, \lambda) =  
\begin{pmatrix}
\cos(\theta/2) & -e^{i\lambda}\sin(\theta/2)\\
e^{i\phi}\sin(\theta/2) & e^{i(\phi + \lambda)}\cos(\theta/2)
\end{pmatrix}.
$$

Notice that the above unitaries are essentially equivalent to $e^{-i{(\phi + \lambda)}/{2}} u3(\theta, \phi, \lambda)$, as in Eq.(2) of the openqasm [here](https://github.com/QISKit/openqasm/blob/master/spec/qasm2.pdf). 

Here is the command to add such a $u3$ gate to a circuit.  

In [2]:
theta = pi/2.0   
phi = pi/3.0
lambd = pi/4.0
demoCircuit.u3(theta, phi, lambd, qr[0]) #applying a u3 gate to the first qubit 

#### u2 (or, U gates with two parameters)

$u2(\phi, \lambda)$ gates are rotations of 1-qubit state whose mathematical definitions are:

$$
u2(\phi, \lambda) = u3(\pi/2, \phi, \lambda) = 
\frac{1}{\sqrt{2}} \begin{pmatrix}
1 & -e^{i\lambda} \\
e^{i\phi} & e^{i(\phi + \lambda)}
\end{pmatrix}.
$$

We can add a $u2$ gate to a circuit as follows.

In [3]:
phi = pi/3.0
lambd = pi/4.0
demoCircuit.u2(phi, lambd, qr[0]) #applying a u2 gate to the first qubit 

#### u1 (or, U gates with one parameter)

$u1(\lambda)$ gates are rotations of 1-qubit state whose mathematical definitions are:

$$
u1(\lambda) = u3(0, 0, \lambda) = 
\begin{pmatrix}
1 & 0 \\
0 & e^{i \lambda}
\end{pmatrix}.
$$

We can add a $u1$ gate to a circuit as below.

In [4]:
lambd = pi/4.0
demoCircuit.u1(lambd, qr[0]) #applying a u1 gate to the first qubit 

#### Hadamard gate (Clifford)

$$
H = u2(0, \pi) =
1/\sqrt{2}
\begin{pmatrix}
1 & 1\\
1 & -1
\end{pmatrix}
$$

#### $S$ (or, $\sqrt{Z}$ phase) gate (Clifford)

$$
S = u1(\pi/2) = 
\begin{pmatrix}
1 & 0\\
0 & e^{i\pi/2}
\end{pmatrix}
$$

#### $S^{\dagger}$ (or, conjugate of $\sqrt{Z}$ phase) gate (Clifford)

$$
S^{\dagger} = u1(-\pi/2) = 
\begin{pmatrix}
1 & 0\\
0 & e^{-i\pi/2}
\end{pmatrix}
$$

The above Clifford gates can be added as below.

In [7]:
demoCircuit.h(qr[0]) #applying h gate to the first qubit
demoCircuit.s(qr[0]) #applying s gate to the first qubit
demoCircuit.sdg(qr[0]) #applying sdg gate to the first qubit

#### $T$ (or, $\sqrt{S}$ phase) gate ($C3$ gate)

$$
T = u1(pi/4) = 
\begin{pmatrix}
1 & 0\\
0 & e^{\pi/4}
\end{pmatrix}
$$

#### $T^{\dagger}$ (or, conjugate of $\sqrt{S}$ phase) gate ($C3$ gate)

$$
T^{\dagger} = u1(-pi/4) = 
\begin{pmatrix}
1 & 0\\
0 & e^{-\pi/4}
\end{pmatrix}
$$

They can be added as below.

In [8]:
demoCircuit.t(qr[0]) #applying t gate to the first qubit
demoCircuit.tdg(qr[0]) #applying tdg gate to the first qubit

### Standard Rotations

The standard rotation gates are those that define rotations around standard axis. 

#### Rotation around X-axis

$$
R_x(\theta) = u3(\theta, -\pi/2, \pi/2) = 
\begin{pmatrix}
\cos(\theta/2) & -i\sin(\theta/2)\\
-i\sin(\theta/2) & \cos(\theta/2).
\end{pmatrix}
$$

#### Rotation round Y-axis

$$
R_y(\theta) = u3(\theta, 0, 0) = 
\begin{pmatrix}
\cos(\theta/2) & \sin(\theta/2)\\
\sin(\theta/2) & \cos(\theta/2).
\end{pmatrix}
$$

#### Rotation around Z-axis

$$
R_z(\phi) = u1(\phi) = 
\begin{pmatrix}
1 & 0 \\
0 & e^{i \phi}
\end{pmatrix}
$$

They can be added as below.

In [9]:
theta = pi/4
phi = pi/4
demoCircuit.rx(theta, qr[0]) #applying rotation around x-axis gate to the first qubit
demoCircuit.ry(theta, qr[0]) #applying rotation around y-axis gate to the first qubit
demoCircuit.rz(phi, qr[0]) #applying rotation around z-axis gate to the first qubit

## Multi-Qubit Gates

The standard multi-qubit gates consist of two-qubit gates and three-qubit gates. The two-qubit gates consist of the controlled Pauli gates, the controlled Hadamard gate, the controlled rotation gates, and the swap gate. The three-qubit gates consist of the Toffoli gate and the Fredkin gate. 

### Controlled Pauli Gates

#### Controlled X (or, controlled NOT) gate
The control-not gate is the following unitary:

$$
C_X = 
\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 1\\
0 & 0 & 1 & 0
\end{pmatrix}. 
$$

#### Controlled $Y$ gate

Apply $Y$ gate to the target qubit if the control qubit is $1$.

$$
C_Y = 
\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & -i\\
0 & 0 & i & 0
\end{pmatrix}
$$

#### Controlled $Z$ (or, controlled Phase) gate

The controlled Z gate flips the phase of the target qubit if the control qubit is $1$.  

$$
C_Z = 
\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & -1
\end{pmatrix}
$$

### Controlled Hadamard gate

Apply $H$ gate to the target qubit if the control qubit is $1$. 

$$
C_H = 
\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}}\\
0 & 0 & \frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{2}}
\end{pmatrix}
$$

In [5]:
demoCircuit.cx(qr[0], qr[1]) #applying cnot gate
demoCircuit.cy(qr[0], qr[1]) #controlled-y
demoCircuit.cz(qr[0], qr[1]) #controlled-z
demoCircuit.ch(qr[0], qr[1]) #controlled-h

### Controlled rotation gates

#### Controlled rotation around Z-axis

Perform rotation around Z-axis on the target qubit if the control qubit is $1$.

$$
C_{RZ}(\lambda) = 
\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & e^{-i\lambda/2} & 0\\
0 & 0 & 0 & e^{i\lambda/2}
\end{pmatrix}
$$

#### Controlled phase rotation

Perform $u1$ on the target qubit if the control qubit is $1$.

$$
C_{u1}(\lambda) = 
\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & e^{i\lambda}
\end{pmatrix}
$$

#### Controlled $U$ rotation

Perform controlled-$U$ rotation on the target qubit if the control qubit is $1$. 

$$
C_{u3}(\theta, \phi, \lambda) = 
\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & \cos(\theta/2) & -e^{i\lambda}\sin(\theta/2)\\
0 & 0 & e^{i\phi}\sin(\theta/2) & e^{i(\phi + \lambda)}\cos(\theta/2)
\end{pmatrix}.
$$

They can be added to the circuit as below.

In [12]:
demoCircuit.crz(lambd, qr[0], qr[1]) #controlled rotation around-Z
demoCircuit.cu1(lambd, qr[0], qr[1]) #controlled u1
demoCircuit.cu3(theta, phi, lambd, qr[0], qr[1]) #controlled u3

### Swap gate

The swap gate exchange the first and second qubits. 

$$
{SWAP} = 
\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 1
\end{pmatrix}.
$$

In [8]:
demoCircuit.swap(qr[0], qr[1])              #swapping the first and second qubits

### Toffoli gate ($C3$ gate)

The Toffoli gate flips the third qubit if the first two qubits are both $1$.

$$
C_{CX} = 
\begin{pmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0
\end{pmatrix}
$$

### Controlled swap gate (Fredkin Gate)

The controlled swap gate exchange the second and third qubits if the first qubit is $1$.

$$
C_{SWAP} = 
\begin{pmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1
\end{pmatrix}
$$

In [9]:
demoCircuit.ccx(qr[0], qr[1], qr[2])        #Toffoli gate
demoCircuit.cswap(qr[0], qr[1], qr[2])      #swapping the second and third qubits controlled by the first qubit